# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [26]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks=pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2166493831492,
 'week52high': 151.43,
 'week52low': 78.28,
 'week52highSplitAdjustOnly': 149.66,
 'week52lowSplitAdjustOnly': 78.1,
 'week52change': 0.6786355535950926,
 'sharesOutstanding': 17225824076,
 'float': 0,
 'avg10Volume': 99835907,
 'avg30Volume': 98062068,
 'day200MovingAvg': 132.5,
 'day50MovingAvg': 130.19,
 'employees': 154091,
 'ttmEPS': 4.49,
 'ttmDividendRate': 0.8500071514430982,
 'dividendYield': 0.006613300262052512,
 'nextDividendDate': '0',
 'exDividendDate': '2021-05-01',
 'nextEarningsDate': '2021-07-16',
 'peRatio': 28.30048239261366,
 'beta': 1.3888966990040725,
 'maxChangePercent': 48.84931839904418,
 'year5ChangePercent': 5.209140119669005,
 'year2ChangePercent': 1.8327501271962139,
 'year1ChangePercent': 0.674930741150863,
 'ytdChangePercent': -0.0368426653825235,
 'month6ChangePercent': 0.07405559600876374,
 'month3ChangePercent': -0.0588036773386456,
 'month1ChangePercent': -0.03415213067615473,
 'day30ChangePer

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.674930741150863

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
            index=my_columns
        ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137.31,0.637137,N/A
1,AAL,22.40,1.50614,N/A
2,AAP,205.66,0.704517,N/A
3,AAPL,127.73,0.676496,N/A
4,ABBV,119.10,0.363707,N/A
...,...,...,...,...
500,YUM,122.26,0.46484,N/A
501,ZBH,170.92,0.518294,N/A
502,ZBRA,507.17,1.25265,N/A
503,ZION,60.90,1.27578,N/A



## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending=False, inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,70.81,5.81493,N/A
1,179,FCX,43.01,4.31619,N/A
2,208,GPS,37.25,4.13143,N/A
3,253,IVZ,28.35,3.25382,N/A
4,441,TPR,48.24,2.78699,N/A
5,272,KSS,61.69,2.73141,N/A
6,314,MOS,36.80,2.62911,N/A
7,410,SIVB,591.42,2.35145,N/A
8,385,PWR,100.59,2.16487,N/A
9,129,DFS,118.77,2.15853,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size=input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [9]:
position_size=float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy']= math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

C:\Users\abhin\anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,70.81,5.81493,2824
1,179,FCX,43.01,4.31619,4650
2,208,GPS,37.25,4.13143,5369
3,253,IVZ,28.35,3.25382,7054
4,441,TPR,48.24,2.78699,4145
5,272,KSS,61.69,2.73141,3242
6,314,MOS,36.80,2.62911,5434
7,410,SIVB,591.42,2.35145,338
8,385,PWR,100.59,2.16487,1988
9,129,DFS,118.77,2.15853,1683


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [24]:
hqm_columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    
]
hqm_dataframe=pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe=hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
            ],
            index=hqm_columns
            ),
            ignore_index=True
        )
        
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True) 
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,134.42,N/A,0.619589,N/A,0.197160,N/A,0.027401,N/A,-0.006292,N/A
1,AAL,23.00,N/A,1.517324,N/A,0.867319,N/A,0.301231,N/A,0.001360,N/A
2,AAP,207.02,N/A,0.706797,N/A,0.329449,N/A,0.333926,N/A,0.081014,N/A
3,AAPL,131.27,N/A,0.690761,N/A,0.073222,N/A,-0.057902,N/A,-0.033489,N/A
4,ABBV,116.55,N/A,0.370981,N/A,0.209607,N/A,0.132166,N/A,0.101159,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.37,N/A,0.471978,N/A,0.173703,N/A,0.137281,N/A,0.009521,N/A
501,ZBH,167.92,N/A,0.526303,N/A,0.110689,N/A,0.039483,N/A,-0.006268,N/A
502,ZBRA,506.34,N/A,1.238822,N/A,0.421652,N/A,0.018923,N/A,-0.028515,N/A
503,ZION,61.90,N/A,1.276663,N/A,0.634006,N/A,0.232018,N/A,0.077278,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [27]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.516832
1      0.918812
2      0.570297
3      0.566337
4       0.30297
         ...   
500    0.386139
501    0.435644
502    0.839604
503    0.863366
504    0.316832
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.467327
1      0.974257
2      0.655446
3      0.219802
4       0.50099
         ...   
500    0.425743
501     0.29703
502    0.764356
503    0.906931
504    0.164356
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.229703
1       0.918812
2       0.940594
3      0.0990099
4       0.510891
         ...    
500     0.528713
501     0.257426
502          0.2
503          0.8
504     0.251485
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.243564
1      0.283168
2      0.734653
3      0.146535
4      0.843564
         ...   
500    0.324752
501    0.245545
502    0.178218
503    0.722772
504    0.625743
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,134.42,N/A,0.619589,0.516832,0.197160,0.467327,0.027401,0.229703,-0.006292,0.243564
1,AAL,23.00,N/A,1.517324,0.918812,0.867319,0.974257,0.301231,0.918812,0.001360,0.283168
2,AAP,207.02,N/A,0.706797,0.570297,0.329449,0.655446,0.333926,0.940594,0.081014,0.734653
3,AAPL,131.27,N/A,0.690761,0.566337,0.073222,0.219802,-0.057902,0.0990099,-0.033489,0.146535
4,ABBV,116.55,N/A,0.370981,0.30297,0.209607,0.50099,0.132166,0.510891,0.101159,0.843564
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.37,N/A,0.471978,0.386139,0.173703,0.425743,0.137281,0.528713,0.009521,0.324752
501,ZBH,167.92,N/A,0.526303,0.435644,0.110689,0.29703,0.039483,0.257426,-0.006268,0.245545
502,ZBRA,506.34,N/A,1.238822,0.839604,0.421652,0.764356,0.018923,0.2,-0.028515,0.178218
503,ZION,61.90,N/A,1.276663,0.863366,0.634006,0.906931,0.232018,0.8,0.077278,0.722772


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [28]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [29]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [30]:
portfolio_input()

Enter the size of your portfolio:10000000


In [31]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\abhin\anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,134.42,1458,0.619589,0.516832,0.197160,0.467327,0.027401,0.229703,-0.006292,0.243564,0.364356
1,AAL,23.00,8525,1.517324,0.918812,0.867319,0.974257,0.301231,0.918812,0.001360,0.283168,0.773762
2,AAP,207.02,947,0.706797,0.570297,0.329449,0.655446,0.333926,0.940594,0.081014,0.734653,0.725248
3,AAPL,131.27,1493,0.690761,0.566337,0.073222,0.219802,-0.057902,0.0990099,-0.033489,0.146535,0.257921
4,ABBV,116.55,1682,0.370981,0.30297,0.209607,0.50099,0.132166,0.510891,0.101159,0.843564,0.539604
5,ABC,117.30,1671,0.432244,0.354455,0.086396,0.241584,0.115696,0.475248,-0.009216,0.239604,0.327723
6,ABMD,270.79,724,0.466931,0.382178,-0.013092,0.0811881,-0.186503,0.0316832,-0.189103,0.0039604,0.124752
7,ABT,118.55,1653,0.315919,0.249505,0.060033,0.19802,-0.076361,0.0871287,-0.026643,0.182178,0.179208
8,ACN,293.80,667,0.627819,0.520792,0.206671,0.491089,0.127652,0.50099,0.013465,0.340594,0.463366
9,ADBE,490.61,399,0.385141,0.318812,0.038173,0.158416,-0.025433,0.132673,-0.047964,0.10099,0.177723


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [32]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [35]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [36]:
writer.save()